<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/DcCup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import datetime
import warnings
from collections import Counter
from imblearn.over_sampling import SMOTE
# import lightgbm as lgb
import xgboost as xgb
# import catboost as cat

warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## 读取文件&合并

In [0]:
data1 = pd.read_csv('/content/drive/My Drive/比赛/dccup/round1_diac2019_train.csv')
print(data1.shape)
data2 = pd.read_csv('/content/drive/My Drive/比赛/dccup/round2_diac2019_train.csv')
print(data1.columns)
print(data2.columns)
data2 = data2.drop(columns=['goods_class_id'])
print(data2.shape)

(1400831, 28)
Index(['order_detail_id', 'order_id', 'order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount', 'customer_province',
       'customer_city', 'member_id', 'customer_id', 'customer_gender',
       'member_status', 'is_member_actived', 'goods_id', 'goods_price',
       'goods_status', 'goods_has_discount', 'goods_list_time',
       'goods_delist_time'],
      dtype='object')
Index(['order_detail_id', 'order_id', 'order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount', 'customer_province',
       'cu

In [0]:
data = pd.concat([data1,data2],axis=0)

In [0]:
def featureDF(dataset,online=False,end_str='2013-11-30',stop_str='2013-12-31'):
    """
    online : 是否线上提交
    :return: 特征df，最后一行为label
    """
    begin_day = datetime.datetime.strptime('2012-11-01', '%Y-%m-%d')
    stop_day = datetime.datetime.strptime(stop_str, '%Y-%m-%d')
    end_day = datetime.datetime.strptime(end_str, '%Y-%m-%d')

    print('加载数据。。。')
    train = dataset

    train = train.fillna(0.0)

    province = list(set(train.customer_province))
    city = list(set(train.customer_city))
    print('Province len:',len(province))
    province = province[1:]
    province.append('0.0')

    print('City len:',len(city))
    city = city[1:]
    city.append('0.0')

    print('处理中。。。')
    train = train.drop(columns=['order_detail_id'])
    train['order_pay_time'] = pd.to_datetime(train['order_pay_time'])
    train['goods_list_time'] = pd.to_datetime(train['goods_list_time'])
    train['goods_delist_time'] = pd.to_datetime(train['goods_delist_time'])

    train['order_pay_time'] = train['order_pay_time'].dt.date
    train['goods_list_time'] = train['goods_list_time'].dt.date
    train['goods_delist_time'] = train['goods_delist_time'].dt.date

    if not online:
        #划分线下数据集
        train_temp = train[(train['order_pay_time'] <= end_day.date())]
        labels = list(set(train[(train['order_pay_time'] > end_day.date()) & (train['order_pay_time'] <= stop_day.date())]['customer_id'].astype(int)))
        #线下训练集截至日期
        # end_day = datetime.datetime.strptime('2013-11-30', '%Y-%m-%d')
        train = train_temp
    else:
        labels = []
        #线上训练集截至日期
        # end_day = datetime.datetime.strptime('2013-12-31', '%Y-%m-%d')
    # end_day = datetime.datetime.strptime(end_str, '%Y-%m-%d')

    train['label'] = 0
    train.loc[train['customer_id'].isin(labels),'label'] = 1

    train['order_status'] = train['order_status'].astype('category')
    train['is_customer_rate'] = train['is_customer_rate'].astype('category')
    train['order_detail_goods_num'] = train['order_detail_goods_num'].astype('category')
    train['customer_gender'] = train['customer_gender'].astype('category')
    train['customer_province'] = train['customer_province'].astype('category')
    train['customer_city'] = train['customer_city'].astype('category')
    train['member_status'] = train['member_status'].astype('category')
    train['is_member_actived'] = train['is_member_actived'].astype('category')
    train['goods_status'] = train['goods_status'].astype('category')
    train['goods_has_discount'] = train['goods_has_discount'].astype('category')

    print('构建用户样本')
    data = train

    customer = {}
    i = 0
    for d in data.values:
        id = d[17]
        if id not in customer:
            customer[id] = {}
            customer[id]['days'] = []
            customer[id]['time'] = 0
            customer[id]['order_list'] = []
            customer[id]['order_status'] = []
            customer[id]['order_discount'] = 0
            customer[id]['count'] = 0
            customer[id]['amount'] = 0
            customer[id]['payment'] = 0
            customer[id]['every_payment'] = []
            customer[id]['every_discount'] = []
            customer[id]['rate'] = []
            customer[id]['order_detail_status'] = []
            customer[id]['detail_amount'] = 0
            customer[id]['detail_payment'] = 0
            customer[id]['detail_discount'] = 0
            customer[id]['every_detail_payment'] = []
            customer[id]['every_detail_discount'] = []
            customer[id]['good_total'] = 0
            customer[id]['good_price'] = 0
            customer[id]['good_status'] = []
            customer[id]['good_list'] = []
            customer[id]['good_discount'] = []
            customer[id]['label'] = d[-1]
        order_id = d[0]
        if order_id not in customer[id]['order_list']:
            customer[id]['days'].append(d[5])
            customer[id]['time'] += 1
            customer[id]['order_list'].append(d[0])
            if d[6] not in range(1, 7):
                customer[id]['order_status'].append(0)
            else:
                customer[id]['order_status'].append(d[6])
            customer[id]['order_discount'] += d[4]
            customer[id]['every_discount'].append(d[4])
            customer[id]['count'] += d[7]
            customer[id]['amount'] += d[2]
            customer[id]['payment'] += d[3]
            customer[id]['every_payment'].append(d[3])
            customer[id]['rate'].append(d[8])
        if d[9] not in range(1, 7):
            customer[id]['order_detail_status'].append(0)
        else:
            customer[id]['order_detail_status'].append(d[9])
        customer[id]['detail_amount'] += d[11]
        customer[id]['detail_payment'] += d[12]
        customer[id]['every_detail_payment'].append(d[12])
        customer[id]['detail_discount'] += d[13]
        customer[id]['every_detail_discount'].append(d[13])
        customer[id]['good_total'] += d[10]
        customer[id]['good_price'] += d[22]
        customer[id]['good_list'].append(d[21])
        customer[id]['good_status'].append(d[-5])
        customer[id]['good_discount'].append(d[-4])
        customer[id]['member_status'] = d[19]
        customer[id]['is_active'] = d[20]
        customer[id]['province'] = province.index(str(d[14]))
        customer[id]['city'] = city.index(str(d[15]))
        customer[id]['gender'] = d[18]
        i += 1
        # if i%100000 == 0:
        #     print('构建', i, '个样本')
    print('构建', i, '个样本')
    
    print('对用户特征编码')

    features = []
    for u in customer:
        temp = []
        # ID 0
        temp.append(u)
        loya = 0
        user = customer[u]
        # 性别 1 类别
        temp.append(int(user['gender']))
        if int(user['gender']) != 0:
            loya += 1
        # 省份 2 类别
        temp.append(user['province'])
        if int(user['province']) != 0:
            loya += 1
        # 城市 3 类别
        temp.append(user['city'])
        if int(user['city']) != 0:
            loya += 1
        # 会员状态 4 类别
        temp.append(user['member_status'])
        if int(user['member_status']) != 0:
            loya += 1
        # 是否激活会员 5 类别
        temp.append(user['is_active'])
        if int(user['is_active']) != 0:
            loya += 1
        # 会员信赖度 指标 6
        temp.append(loya/5)
        # 购物次数 7
        temp.append(user['time'])
        # 购买商品件数 8
        temp.append(user['good_total'])
        # 购物总金额 9
        temp.append(user['amount'])
        # 购物支付金额 10
        temp.append(user['payment'])
        # 购物折扣比例 11,12
        if user['amount'] != 0:
            temp.append(user['order_discount'] / user['amount'])
        else:
            temp.append(0)
        if user['payment'] != 0:
            temp.append(user['order_discount'] / user['payment'])
        else:
            temp.append(0)
        # 最后一单付款金额 13
        temp.append(user['every_payment'][-1])
        # 最后一单折扣金额 14
        temp.append(user['every_discount'][-1])
        # 最后一单折扣率 15
        if user['every_payment'][-1] != 0:
            temp.append(user['every_discount'][-1]/user['every_payment'][-1])
        else:
            temp.append(0)
        # 单品最多买次数 16
        # 单品平均购买次数
        total_time = 0
        for p in set(user['good_list']):
            total_time += user['good_list'].count(p)
        temp.append(total_time/len(user['good_list']))
        # 订单状态占比 17,18,19,20,21,22,23
        for s in range(7):
            temp.append(user['order_status'].count(s) / len(user['order_status']))
        # 订单评论占比 24,25
        for r in range(2):
            temp.append(user['rate'].count(r) / len(user['rate']))
        # 子订单状态占比 26,27,28,29,30,31,32
        for s in range(7):
            temp.append(user['order_detail_status'].count(s) / len(user['order_detail_status']))
        # 最后一次订单评论状态 33
        temp.append(user['rate'][-1])
        # 最后一次订单状态 34
        temp.append(user['order_status'][-1])
        # 最后一次子订单状态 35
        temp.append(user['order_detail_status'][-1])
        # 订单总价 新添加 36
        temp.append(user['detail_amount'])
        # 订单实付 新添加 37
        temp.append(user['detail_payment'])
        # 订单折扣 新添加 38
        temp.append(user['detail_discount'])
        # 最后一单付款金额 39
        temp.append(user['every_detail_payment'][-1])
        # 最后一单折扣金额 40
        temp.append(user['every_detail_discount'][-1])
        # 最后一单折扣率 41
        if user['every_detail_payment'][-1] != 0:
            temp.append(user['every_detail_discount'][-1]/user['every_detail_payment'][-1])
        else:
            temp.append(0)
        # 订单折扣占比 新添加 42
        if user['detail_amount'] != 0:
            temp.append(user['detail_discount']/user['detail_amount'])
        else:
            temp.append(0)
        # 商品原始价格 新添加 43
        temp.append(user['good_price'])
        # 原价折扣比例 新添加 44
        if user['good_price'] != 0:
            temp.append(user['detail_amount']/user['good_price'])
        else:
            temp.append(0)
        # 商品状态占比 45,46,47
        for s in range(3):
            temp.append(user['good_status'].count(s) / len(user['good_status']))
        # 商品对会员优惠占比 48,49
        for d in range(2):
            temp.append(user['good_discount'].count(d) / len(user['good_discount']))
        # 第一次购物离开始日天数 50
        temp.append((user['days'][0] - begin_day.date()).days)
        # 最后一次购物离截至日天数 51
        temp.append((end_day.date() - user['days'][-1]).days)
        # 第一次购物距最后一次购物天数 52
        temp.append((user['days'][0] - user['days'][-1]).days)
        # 购物平均间隔天数 53
        mean_day = 0
        for d in range(len(user['days']) - 1, 0, -1):
            mean_day += (user['days'][d] - user['days'][d - 1]).days
        if len(user['days']) > 1:
            temp.append(mean_day / (len(user['days']) - 1))
        else:
            temp.append(0)
        # label 54
        temp.append(user['label'])
        features.append(temp)

    df_feature = pd.DataFrame(features)
    print('数据集形状',df_feature.shape)

    return df_feature

In [0]:
df = featureDF(data,online=False,end_str='2013-11-30',stop_str='2013-12-31')
print('加载完成', df.shape)
df.head()

加载数据。。。
Province len: 35
City len: 380
处理中。。。
构建用户样本
构建 3965326 个样本
对用户特征编码
数据集形状 (1826575, 55)
加载完成 (1826575, 55)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,133.0,0,0,43,0.0,0.0,0.2,3,4.0,864.00,785.80,0.000000,0.000000,105.8,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6,6.0,105.80,105.80,78.20,52.9,39.1,0.739130,0.739130,879.00,0.120364,0.0,0.500000,0.500000,0.000000,1.000000,61,41,-292,146.000000,0
1,413.0,0,25,203,0.0,0.0,0.4,3,3.0,739.90,589.90,0.000000,0.000000,89.9,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6,6.0,89.90,89.90,150.00,89.9,150.0,1.668521,1.668521,979.90,0.091744,0.0,0.333333,0.666667,0.333333,0.666667,61,52,-281,140.500000,0
2,113.0,0,30,251,0.0,0.0,0.4,2,2.0,552.90,552.90,0.000000,0.000000,132.9,0.0,0.0,1.0,0.000000,0.500000,0.0,0.0,0.0,0.0,0.500000,1.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.0,1,1.0,132.90,132.90,0.00,132.9,0.0,0.000000,0.000000,819.90,0.162093,0.0,1.000000,0.000000,0.500000,0.500000,62,40,-292,292.000000,0
3,502.0,0,25,296,0.0,0.0,0.4,4,4.0,887.00,837.90,0.000000,0.000000,52.9,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6,6.0,417.90,417.90,49.10,52.9,49.1,0.928166,0.117492,902.00,0.463304,0.0,0.750000,0.250000,0.250000,0.750000,61,56,-277,92.333333,0
4,0.0,0,18,236,0.0,0.0,0.4,48,48.0,14088.85,13551.81,0.021275,0.022118,96.9,0.0,0.0,1.0,0.020833,0.020833,0.0,0.0,0.0,0.0,0.958333,0.833333,0.166667,0.020833,0.020833,0.0,0.0,0.0,0.0,0.958333,0.0,6,6.0,13781.56,13531.82,29.23,96.9,143.0,1.475748,0.002121,14028.85,0.982373,0.0,0.354167,0.645833,0.125000,0.875000,61,394,61,-1.297872,1


In [0]:
def logloss(y_true, y_pred):
    deta = 3
    eps=1e-15
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred[:,1])
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss/len(y_true)
def singleloss(y_true, y_pred):
    deta = 3
    eps=1e-15
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss/len(y_true)
def evalerror(preds, dtrain):
    deta = 3
    eps=1e-15
    labels = dtrain.get_label()
    y_true = np.array(labels)
    y_pred = np.array(preds)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return 'error', loss/len(y_true)

In [0]:
smo = SMOTE()
x_smo, y_smo = smo.fit_sample(df.values[:,:-1], df.values[:,-1])

In [0]:
print(Counter(y_smo))

Counter({0.0: 1735651, 1.0: 1735651})


In [0]:
train_data = pd.DataFrame(x_smo)
train_data[54] = y_smo
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,133.0,0.0,0.0,43.0,0.0,0.0,0.2,3.0,4.0,864.00,785.80,0.000000,0.000000,105.8,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6.0,6.0,105.80,105.80,78.20,52.9,39.1,0.739130,0.739130,879.00,0.120364,0.0,0.500000,0.500000,0.000000,1.000000,61.0,41.0,-292.0,146.000000,0.0
1,413.0,0.0,25.0,203.0,0.0,0.0,0.4,3.0,3.0,739.90,589.90,0.000000,0.000000,89.9,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6.0,6.0,89.90,89.90,150.00,89.9,150.0,1.668521,1.668521,979.90,0.091744,0.0,0.333333,0.666667,0.333333,0.666667,61.0,52.0,-281.0,140.500000,0.0
2,113.0,0.0,30.0,251.0,0.0,0.0,0.4,2.0,2.0,552.90,552.90,0.000000,0.000000,132.9,0.0,0.0,1.0,0.000000,0.500000,0.0,0.0,0.0,0.0,0.500000,1.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.0,1.0,1.0,132.90,132.90,0.00,132.9,0.0,0.000000,0.000000,819.90,0.162093,0.0,1.000000,0.000000,0.500000,0.500000,62.0,40.0,-292.0,292.000000,0.0
3,502.0,0.0,25.0,296.0,0.0,0.0,0.4,4.0,4.0,887.00,837.90,0.000000,0.000000,52.9,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,6.0,6.0,417.90,417.90,49.10,52.9,49.1,0.928166,0.117492,902.00,0.463304,0.0,0.750000,0.250000,0.250000,0.750000,61.0,56.0,-277.0,92.333333,0.0
4,0.0,0.0,18.0,236.0,0.0,0.0,0.4,48.0,48.0,14088.85,13551.81,0.021275,0.022118,96.9,0.0,0.0,1.0,0.020833,0.020833,0.0,0.0,0.0,0.0,0.958333,0.833333,0.166667,0.020833,0.020833,0.0,0.0,0.0,0.0,0.958333,0.0,6.0,6.0,13781.56,13531.82,29.23,96.9,143.0,1.475748,0.002121,14028.85,0.982373,0.0,0.354167,0.645833,0.125000,0.875000,61.0,394.0,61.0,-1.297872,1.0


In [0]:
train_x,valid_x,train_y,valid_y = train_test_split(train_data[:,:-1],train_data[:,-1],test_size=0.1,striafy=train_data[:,-1])
xgb_train = xgb.DMatrix(train_x,train_y)
xgb_valid = xgb.DMatrix(valid_x,valid_y)
model = xgb.train(params,dtrain=xgb_train,num_boost_round=2000,early_stopping_rounds=20,evals=evallist)

In [0]:
train_data = shuffle(train_data)

In [0]:
train_data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54],
           dtype='int64')

In [0]:
params = {
    'objective':'binary:logistic',
    'learning_rate': 0.01,
    'min_child_weight': 2,
    'max_depth': 12,
    'gamma': 0.8,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'colsample_bylevel': 0.9,
    'reg_alpha': 0.1,
}

In [0]:
def XGBKfoldResult(train=train_data.iloc[:,:-1],label=train_data.iloc[:,-1],params=params,K=5):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  # test = xgb.DMatrix(test.copy())
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      xgb_train = xgb.DMatrix(train_x,train_y)
      xgb_valid = xgb.DMatrix(valid_x,valid_y)
      evallist = [(xgb_valid,'eval')]
      model = xgb.train(params,dtrain=xgb_train,num_boost_round=2000,early_stopping_rounds=20,evals=evallist)
      models.append(model)
      # pred = model.predict(test)
      # preds.append(pred)

  return models,pret)

In [0]:
models,preds = XGBKfoldResult()

第 0 个Fold
[0]	eval-error:0.499621
Will train until eval-error hasn't improved in 20 rounds.
[1]	eval-error:0.500336
[2]	eval-error:0.499928
[3]	eval-error:0.50051
[4]	eval-error:0.500449
[5]	eval-error:0.500592
[6]	eval-error:0.500333
[7]	eval-error:0.500285
[8]	eval-error:0.500994
[9]	eval-error:0.500854
[10]	eval-error:0.501033
[11]	eval-error:0.500917
[12]	eval-error:0.500609
[13]	eval-error:0.500578
[14]	eval-error:0.500544
[15]	eval-error:0.50056
[16]	eval-error:0.500542
[17]	eval-error:0.500974
[18]	eval-error:0.500871
[19]	eval-error:0.50065
[20]	eval-error:0.500742
Stopping. Best iteration:
[0]	eval-error:0.499621

第 1 个Fold
[0]	eval-error:0.500029
Will train until eval-error hasn't improved in 20 rounds.
[1]	eval-error:0.499585
[2]	eval-error:0.499885
[3]	eval-error:0.499902
[4]	eval-error:0.499758
[5]	eval-error:0.499674
[6]	eval-error:0.49998
[7]	eval-error:0.500508
[8]	eval-error:0.500408
[9]	eval-error:0.500343
[10]	eval-error:0.500482
[11]	eval-error:0.500657
[12]	eval-er

In [0]:
import pickle as pkl

In [0]:
with open('models.pkl','wb') as source:
  pkl.dump(models,source)

#### 生成全量预测集合

In [0]:
with open('models.pkl','rb') as source:
  models = pkl.load(source)

In [0]:
test_data = featureDF(data,online=True,end_str='2013-12-31')

加载数据。。。
Province len: 35
City len: 380
处理中。。。
构建用户样本
构建 4086331 个样本
对用户特征编码
数据集形状 (1826575, 55)


In [0]:
test_data = test_data.drop(columns=[54])

KeyError: ignored

In [0]:
test = xgb.DMatrix(test_data)

In [0]:
out = []
for model in models:
  out.append(model.predict(test))
out = np.array(out).mean(axis=0)

In [0]:
test_data['predict'] = out

In [0]:
output = test_data[[0,'predict']]
output = output.rename({0:'customer_id','predict':'result'},axis=1)
output.to_csv('reuslt.csv',index=False)

In [0]:
output.shape

(1826575, 2)

In [0]:
output

,customer_id,result
0,133.0,0.499389
1,413.0,0.500594
2,113.0,0.501113
3,502.0,0.500109
4,0.0,0.500604
...,...,...
1826570,1826570.0,0.501140
1826571,1826571.0,0.497674
1826572,1826572.0,0.499534
1826573,1826573.0,0.503504
